## For this example (move base):
<img src = "Pyomo_Diagram.png" width = "500">

In [1]:
%reset # clears variables in workspace

# import libraries
import sympy as sym
import numpy as np

from IPython.display import display
from sympy import pprint

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Don't know how to reset  #, please run `%reset?` for details
Don't know how to reset  clears, please run `%reset?` for details
Don't know how to reset  variables, please run `%reset?` for details
Flushing input history
Don't know how to reset  workspace, please run `%reset?` for details


In [2]:
'''create symbolic variables'''

# system parameters
g = sym.symbols('g')
mu = 150*(np.pi/180)

# mass of links
m_ull,m_lll = sym.symbols(['m_ull','m_lll'])
m_url,m_lrl = sym.symbols(['m_url','m_lrl'])
m_body,m_foot = sym.symbols(['m_body','m_foot'])
m_boom = sym.symbols('m_boom')

# length of links
l_foot,l_body,LL = sym.symbols(['l_foot','l_body','LL'])
l_ull,l_lll = sym.symbols(['l_ull','l_lll'])
l_url,l_lrl = sym.symbols(['l_url','l_lrl'])

# Moment of inertia of links
In_body = sym.symbols('In_body')
In_ull = sym.symbols('In_ull')
In_lll = sym.symbols('In_lll')
In_url = sym.symbols('In_url')
In_lrl = sym.symbols('In_lrl')
In_foot = sym.symbols('In_foot')

# Angles, Velocities and Accelerations
x_body,y_body = sym.symbols(['x_body','y_body'])
th_body = sym.symbols('theta_body') 
th_ull,th_lll = sym.symbols(['theta_ull','theta_lll'])
th_url,th_lrl = sym.symbols(['theta_url','theta_lrl'])

dx_body,dy_body = sym.symbols(['\dot{\\x}_{body}','\dot{\\y}_{body}'])
dth_body = sym.symbols('\dot{\\theta}_{body}') 
dth_ull,dth_lll = sym.symbols(['\dot{\\theta}_{ull}','\dot{\\theta}_{lll}'])
dth_url,dth_lrl = sym.symbols(['\dot{\\theta}_{url}','\dot{\\theta}_{lrl}'])

ddx_body,ddy_body = sym.symbols(['\ddot{\\x}_{body}','\ddot{\\y}_{body}'])
ddth_body = sym.symbols('\ddot{\\theta}_{body}') 
ddth_ull,ddth_lll = sym.symbols(['\ddot{\\theta}_{ull}','\ddot{\\theta}_{lll}'])
ddth_url,ddth_lrl = sym.symbols(['\ddot{\\theta}_{url}','\ddot{\\theta}_{lrl}'])

# Generalized Coordinates
q = sym.Matrix([[x_body],[y_body],[th_body],[th_ull],[th_lll],[th_url],[th_lrl]]) #group into matrices
dq = sym.Matrix([[dx_body],[dy_body],[dth_body],[dth_ull],[dth_lll],[dth_url],[dth_lrl]])
ddq = sym.Matrix([[ddx_body],[ddy_body],[ddth_body],[ddth_ull],[ddth_lll],[ddth_url],[ddth_lrl]])

In [3]:
''' Setup positions (to COM) and velocities'''

height = 2/3

# Absolute angles
th_LU = th_body + th_ull
th_LL = th_lll - (np.pi - th_LU)
th_RU = th_body + th_url
th_RL = th_lrl - (np.pi - th_RU)

# position to COM of each link
r_body = sym.Matrix([[x_body,
                      y_body]]).transpose()

r_boom = sym.Matrix([[x_body*height,
                      y_body*height]]).transpose()

r_ull = sym.Matrix([[x_body-LL/2*sym.cos(th_body)-l_ull/2*sym.cos(th_LU),
                     y_body-LL/2*sym.sin(th_body)-l_ull/2*sym.sin(th_LU)]]).transpose()

r_lll = sym.Matrix([[x_body-LL/2*sym.cos(th_body)-l_ull*sym.cos(th_LU)+l_lll/2*sym.cos(np.pi-th_LL),
                     y_body-LL/2*sym.sin(th_body)-l_ull*sym.sin(th_LU)-l_lll/2*sym.sin(np.pi-th_LL)]]).transpose()

r_url = sym.Matrix([[x_body+LL/2*sym.cos(th_body)+l_url/2*sym.cos(np.pi-th_RU),
                     y_body+LL/2*sym.sin(th_body)-l_url/2*sym.sin(np.pi-th_RU)]]).transpose()

r_lrl = sym.Matrix([[x_body+LL/2*sym.cos(th_body)+l_url*sym.cos(np.pi-th_RU)-l_lrl/2*sym.cos(th_RL),
                     y_body+LL/2*sym.sin(th_body)-l_url*sym.sin(np.pi-th_RU)-l_lrl/2*sym.sin(th_RL)]]).transpose()

r_foot = sym.Matrix([[x_body+LL/2*sym.cos(th_body)+l_url*sym.cos(np.pi-th_RU)-l_lrl*sym.cos(th_RL)+l_foot/2*sym.cos(mu-th_RL),
                      y_body+LL/2*sym.sin(th_body)-l_url*sym.sin(np.pi-th_RU)-l_lrl*sym.sin(th_RL)-l_foot/2*sym.sin(mu-th_RL)]]).transpose()

# translational velocity
dr_body = r_body.jacobian(q)*dq
dr_boom = r_boom.jacobian(q)*dq

dr_ull = r_ull.jacobian(q)*dq
dr_lll = r_lll.jacobian(q)*dq

dr_url = r_url.jacobian(q)*dq
dr_lrl = r_lrl.jacobian(q)*dq

dr_foot = r_foot.jacobian(q)*dq

# angular velocity
w_body = dth_body
w_ull = dth_ull + dth_body
w_lll = dth_ull + dth_lll + dth_body
w_url = dth_url + dth_body
w_lrl = dth_url + dth_lrl + dth_body
w_foot = w_lrl

In [4]:
# Kinetic Energy
T_body = 0.5*sym.Matrix([[m_body,m_body]])*sym.matrix_multiply_elementwise(dr_body,dr_body) + sym.Matrix([[0.5*w_body*w_body*In_body]])

T_boom = 0.5*sym.Matrix([[m_boom,m_boom]])*sym.matrix_multiply_elementwise(dr_boom,dr_boom)

T_ull = 0.5*sym.Matrix([[m_ull,m_ull]])*sym.matrix_multiply_elementwise(dr_ull,dr_ull) + sym.Matrix([[0.5*w_ull*w_ull*In_ull]])

T_lll = 0.5*sym.Matrix([[m_lll,m_lll]])*sym.matrix_multiply_elementwise(dr_lll,dr_lll) + sym.Matrix([[0.5*w_lll*w_lll*In_lll]])

T_url = 0.5*sym.Matrix([[m_url,m_url]])*sym.matrix_multiply_elementwise(dr_url,dr_url) + sym.Matrix([[0.5*w_url*w_url*In_url]])

T_lrl = 0.5*sym.Matrix([[m_lrl,m_lrl]])*sym.matrix_multiply_elementwise(dr_lrl,dr_lrl) + sym.Matrix([[0.5*w_lrl*w_lrl*In_lrl]])

T_foot = 0.5*sym.Matrix([[m_foot,m_foot]])*sym.matrix_multiply_elementwise(dr_foot,dr_foot) + sym.Matrix([[0.5*w_foot*w_foot*In_foot]])

Ttot = T_body + T_boom + T_ull + T_lll + T_url + T_lrl + T_foot
Ttot = Ttot[0] # since adding all the kinetic energies gives a 1x1 matrix

# Potential Energy
V_body = m_body*g*r_body[1]
V_boom = m_boom*g*r_boom[1]
V_ull = m_ull*g*r_ull[1]
V_lll = m_lll*g*r_lll[1]
V_url = m_url*g*r_url[1]
V_lrl = m_lrl*g*r_lrl[1]
V_foot = m_foot*g*r_foot[1]

Vtot = V_body+V_boom+V_ull+V_lll+V_url+V_lrl+V_foot

In [5]:
# Manipulator Equation (M*ddq + C + G = Tau + Q)

# M Matrix
M = sym.zeros(len(q),len(q))

for i in range(len(q)):
    for j in range(len(q)):
        M[i,j] = sym.diff(sym.diff(Ttot,dq[i]),dq[j])

M = sym.simplify(M)

M_size = int(np.sqrt(len(M))) # assumes square matrix (len(M) = 4x4 = 16)

dM = sym.zeros(M_size,M_size)

for i in range(M_size):
    for j in range(M_size):
        dM[i,j] = sym.Matrix([M[i,j]]).jacobian(q)*dq

# C Matrix
C = dM*dq - (sym.Matrix([Ttot]).jacobian(q)).transpose() # 4x1 - 1x4^Transpose

# G Matrix
G = (sym.Matrix([Vtot]).jacobian(q)).transpose() # 4x1

tau_left, tau_right = sym.symbols(['tau_left','tau_right']) # torques
lamda_x, lamda_y = sym.symbols(['lamda_x','lamda_y']) # contact force (hold links)
GRF_x, GRF_y = sym.symbols(['GRF_x','GRF_y'])

# Torque
Tau = sym.Matrix([[0],[0],[0],[tau_left],[0],[tau_right],[0]]) # forces in the [[th_ull],[th_lll],[th_url],[th_lrl]]

# connection point from left and right links
connect_position_left = sym.Matrix([[x_body-LL/2*sym.cos(th_body)-l_ull*sym.cos(th_LU)+l_lll*sym.cos(np.pi-th_LL)],
                                 [y_body-LL/2*sym.sin(th_body)-l_ull*sym.sin(th_LU)-l_lll*sym.sin(np.pi-th_LL)]]).transpose()

connect_position_right = sym.Matrix([[x_body+LL/2*sym.cos(th_body)+l_url*sym.cos(np.pi-th_RU)-l_lrl*sym.cos(th_RL)],
                                  [y_body+LL/2*sym.sin(th_body)-l_url*sym.sin(np.pi-th_RU)-l_lrl*sym.sin(th_RL)]]).transpose()

# foot position
foot_position = sym.Matrix([[x_body+LL/2*sym.cos(th_body)+l_url*sym.cos(np.pi-th_RU)-l_lrl*sym.cos(th_RL)+l_foot*sym.cos(mu-th_RL)],
                            [y_body+LL/2*sym.sin(th_body)-l_url*sym.sin(np.pi-th_RU)-l_lrl*sym.sin(th_RL)-l_foot*sym.sin(mu-th_RL)]]).transpose()

Jleft = connect_position_left.jacobian(q)
Jright = connect_position_right.jacobian(q)
Jfoot = foot_position.jacobian(q)
lamda_GRF = sym.Matrix([[GRF_x],[GRF_y]])

# forces to hold links together
lamda_connect_left = sym.Matrix([[-lamda_x],[-lamda_y]])
lamda_connect_right = sym.Matrix([[lamda_x],[lamda_y]])

Q = (Jfoot.transpose()*lamda_GRF) + (Jleft.transpose()*lamda_connect_left) + (Jright.transpose()*lamda_connect_right) # Force*partial(position_to_force,q)

In [6]:
# Manipulator Equation 
EOM = M*ddq + G + C - Tau - Q 

EOMs = sym.zeros(len(q),1)
for j in range(len(q)):
    EOMs[j] = EOM[j].simplify()

In [7]:
# save the EOM in a file (saving the EOM saves lots of time)
import dill
dill.dump_session('SAVE_EOM.db')